In [1]:
import pandas as pd
import numpy as np
import re

Read the dataset

In [2]:
data_draft_history = pd.read_csv('revised_player_draft_history.csv')
data_player_info = pd.read_csv('player_info_raw.csv')

In [3]:
data_draft_history.columns

Index(['DRAFT_ID', 'PLAYER_ID', 'PLAYER_NAME', 'SEASON', 'ROUND_NUMBER',
       'ROUND_PICK', 'OVERALL_PICK', 'TEAM_ID', 'TEAM_CITY', 'TEAM_NAME',
       'TEAM_ABBREVIATION', 'ORGANIZATION', 'ORGANIZATION_TYPE'],
      dtype='object')

In [4]:
data_player_info = data_player_info.drop(data_player_info.columns[0], axis=1)
data_player_info.columns

Index(['FULL_NAME', 'YEAR_START', 'YEAR_END', 'POSITION', 'HEIGHT', 'WEIGHT',
       'BIRTH_DATE', 'COLLEGE'],
      dtype='object')

In [5]:
data_player_info.apply(lambda x: sum(x.isnull()),axis=0)

FULL_NAME     0
YEAR_START    0
YEAR_END      0
POSITION      0
HEIGHT        0
WEIGHT        0
BIRTH_DATE    0
COLLEGE       0
dtype: int64

In [6]:
for i in range(len(data_player_info)):
    name = data_player_info.loc[i, 'FULL_NAME']
    id = None
    for j in range(len(data_draft_history)):
        if name == data_draft_history.loc[j, 'PLAYER_NAME']:
            id = data_draft_history.loc[j, 'PLAYER_ID']
    if id != None:
        data_player_info.loc[i, 'PLAYER_ID'] = id

In [7]:
for i in range(len(data_player_info)):
    name = data_player_info.loc[i, 'FULL_NAME']
    college = data_player_info.loc[i, 'COLLEGE']; college = re.sub(',', '', college)
    id = None
    for j in range(len(data_draft_history)):
        if (name == data_draft_history.loc[j, 'PLAYER_NAME']) and (re.sub('-', ' ', data_draft_history.loc[j, 'ORGANIZATION']) in college):
            id = data_draft_history.loc[j, 'PLAYER_ID']
    if id != None:
        data_player_info.loc[i, 'PLAYER_ID'] = id

In [8]:
print(data_player_info.head(10))

             FULL_NAME  YEAR_START  YEAR_END POSITION HEIGHT  WEIGHT  \
0       Alaa Abdelnaby        1991      1995      F-C   6-10   240.0   
1      Zaid Abdul-Aziz        1969      1978      C-F    6-9   235.0   
2  Kareem Abdul-Jabbar        1970      1989        C    7-2   225.0   
3   Mahmoud Abdul-Rauf        1991      2001        G    6-1   162.0   
4    Tariq Abdul-Wahad        1998      2003        F    6-6   223.0   
5  Shareef Abdur-Rahim        1997      2008        F    6-9   225.0   
6        Tom Abernethy        1977      1981        F    6-7   220.0   
7          Forest Able        1957      1957        G    6-3   180.0   
8       John Abramovic        1947      1948        F    6-3   195.0   
9           Alex Acker        2006      2009        G    6-5   185.0   

          BIRTH_DATE                                COLLEGE  PLAYER_ID  
0      June 24, 1968                        Duke University    76001.0  
1      April 7, 1946                  Iowa State University  

In [9]:
dupl_name = data_draft_history['PLAYER_NAME'].duplicated()
dupl_org = data_draft_history['ORGANIZATION'].duplicated()
set1 = set(data_draft_history.loc[dupl_name, 'PLAYER_NAME']); set1 = list(set1)
print(set1)

We need to revise the data manually, because these three names own the same university but have different id

In [11]:
shoudong = []
for i in set1:
    temp1 = data_draft_history[data_draft_history['PLAYER_NAME'] == i]
    temp1_id_dupl = temp1['PLAYER_ID'].duplicated()
    temp1_org_dupl = temp1['ORGANIZATION'].duplicated()
    if (True not in list(temp1_id_dupl)) and (True in list(temp1_org_dupl)):
        shoudong.append(i)
print(shoudong)

['Glen Rice', 'Tom Brennan', 'Patrick Ewing']


There is only one person in player_info.csv for the name of 'Tom Brennan', we just fill the ID of him by searching the Internet

In [12]:
data_player_info[data_player_info['FULL_NAME'] == shoudong[0]]


,FULL_NAME,YEAR_START,YEAR_END,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,COLLEGE,PLAYER_ID
3120,Glen Rice,1990,2004,F,6-7,215.0,"May 28, 1967",University of Michigan,203318.0
3121,Glen Rice,2014,2015,G,6-6,206.0,"January 1, 1991",Georgia Institute of Technology,203318.0


There are two person for the name of 'Patrick Ewing', so we need to fill the name seperately

In [13]:
data_player_info[data_player_info['FULL_NAME'] == shoudong[1]]


,FULL_NAME,YEAR_START,YEAR_END,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,COLLEGE,PLAYER_ID
413,Tom Brennan,1955,1955,F,6-3,195.0,"August 6, 1930",Villanova University,80014.0


In [14]:
data_draft_history[data_draft_history['PLAYER_NAME'] == shoudong[1]]

,DRAFT_ID,PLAYER_ID,PLAYER_NAME,SEASON,ROUND_NUMBER,ROUND_PICK,OVERALL_PICK,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,ORGANIZATION,ORGANIZATION_TYPE
486,487,76246,Tom Brennan,1952,0,0,0,1610612744,Philadelphia,Warriors,PHW,Villanova,College/University
1136,1137,80014,Tom Brennan,1958,8,5,60,1610612744,Philadelphia,Warriors,PHW,Villanova,College/University


In [15]:
data_player_info.loc[1112, 'PLAYER_ID'] = 121
data_player_info.iloc[[1112]]

,FULL_NAME,YEAR_START,YEAR_END,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,COLLEGE,PLAYER_ID
1112,Patrick Ewing,1986,2002,C-F,7-0,240.0,"August 5, 1962",Georgetown University,121.0


As for the name 'Glen Rice', they are the father and son, so they had the similar attributes

In [16]:
data_player_info[data_player_info['FULL_NAME'] == shoudong[2]]

,FULL_NAME,YEAR_START,YEAR_END,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,COLLEGE,PLAYER_ID
1112,Patrick Ewing,1986,2002,C-F,7-0,240.0,"August 5, 1962",Georgetown University,121.0
1113,Patrick Ewing,2011,2011,F,6-8,235.0,"May 20, 1984",Georgetown University,201607.0


In [17]:
data_draft_history[data_draft_history['PLAYER_NAME'] == shoudong[2]]

,DRAFT_ID,PLAYER_ID,PLAYER_NAME,SEASON,ROUND_NUMBER,ROUND_PICK,OVERALL_PICK,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,ORGANIZATION,ORGANIZATION_TYPE
5661,5662,121,Patrick Ewing,1985,1,1,1,1610612752,New York,Knicks,NYK,Georgetown,College/University
7345,7346,201607,Patrick Ewing,2008,2,13,43,1610612758,Sacramento,Kings,SAC,Georgetown,College/University


In [18]:
data_player_info.loc[3120, 'PLAYER_ID'] = 779
data_player_info.loc[3121, 'PLAYER_ID'] = 203318
data_player_info[data_player_info['FULL_NAME'] == shoudong[2]]

,FULL_NAME,YEAR_START,YEAR_END,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,COLLEGE,PLAYER_ID
1112,Patrick Ewing,1986,2002,C-F,7-0,240.0,"August 5, 1962",Georgetown University,121.0
1113,Patrick Ewing,2011,2011,F,6-8,235.0,"May 20, 1984",Georgetown University,201607.0


In [19]:
data_player_info.apply(lambda x: sum(x.isnull()),axis=0)

FULL_NAME        0
YEAR_START       0
YEAR_END         0
POSITION         0
HEIGHT           0
WEIGHT           0
BIRTH_DATE       0
COLLEGE          0
PLAYER_ID     1127
dtype: int64

There are still some data not contained in draft history data, so we delete it

In [20]:
still_no_match = np.where(pd.isnull(data_player_info['PLAYER_ID']))
data_player_info.iloc[still_no_match[0]]

,FULL_NAME,YEAR_START,YEAR_END,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,COLLEGE,PLAYER_ID
8,John Abramovic,1947,1948,F,6-3,195.0,"February 9, 1919",Salem International University,NaN
10,Don Ackerman,1954,1954,G,6-0,183.0,"September 4, 1930",Long Island University,NaN
12,Bud Acton,1968,1968,F,6-6,210.0,"January 11, 1942",Hillsdale College,NaN
24,Jeff Adrien,2011,2015,F,6-7,245.0,"February 10, 1986",University of Connecticut,NaN
28,Blake Ahearn,2008,2012,G,6-2,190.0,"May 27, 1984",Missouri State University,NaN
32,Josh Akognon,2013,2013,G,5-11,185.0,"February 10, 1986","California State University, Fullerton",NaN
39,Cliff Alexander,2016,2016,F,6-8,245.0,"November 16, 1995",University of Kansas,NaN
42,Gary Alexander,1994,1994,F,6-7,240.0,"November 1, 1969",University of South Florida,NaN
52,Malik Allen,2002,2011,F-C,6-10,255.0,"June 27, 1978",Villanova University,NaN
58,Lance Allred,2008,2008,C,6-11,250.0,"February 2, 1981",Weber State University,NaN


In [21]:
data_player_info = data_player_info.drop(still_no_match[0])
data_player_info

,FULL_NAME,YEAR_START,YEAR_END,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,COLLEGE,PLAYER_ID
0,Alaa Abdelnaby,1991,1995,F-C,6-10,240.0,"June 24, 1968",Duke University,76001.0
1,Zaid Abdul-Aziz,1969,1978,C-F,6-9,235.0,"April 7, 1946",Iowa State University,76002.0
2,Kareem Abdul-Jabbar,1970,1989,C,7-2,225.0,"April 16, 1947","University of California, Los Angeles",76003.0
3,Mahmoud Abdul-Rauf,1991,2001,G,6-1,162.0,"March 9, 1969",Louisiana State University,51.0
4,Tariq Abdul-Wahad,1998,2003,F,6-6,223.0,"November 3, 1974",San Jose State University,1505.0
5,Shareef Abdur-Rahim,1997,2008,F,6-9,225.0,"December 11, 1976",University of California,949.0
6,Tom Abernethy,1977,1981,F,6-7,220.0,"May 6, 1954",Indiana University,76005.0
7,Forest Able,1957,1957,G,6-3,180.0,"July 27, 1932",Western Kentucky University,76006.0
9,Alex Acker,2006,2009,G,6-5,185.0,"January 21, 1983",Pepperdine University,101165.0
11,Mark Acres,1988,1993,F-C,6-11,220.0,"November 15, 1962",Oral Roberts University,76009.0


Recheck the missing of data

In [22]:
data_player_info.apply(lambda x: sum(x.isnull()),axis=0)

FULL_NAME     0
YEAR_START    0
YEAR_END      0
POSITION      0
HEIGHT        0
WEIGHT        0
BIRTH_DATE    0
COLLEGE       0
PLAYER_ID     0
dtype: int64

Save the data

In [23]:
data_player_info.to_csv('revised_player_info.csv', index=False)